In [1]:
# add the root to sys.path
import sys
sys.path.append('/Users/vinz/Documents/ETH/Do4/Simulation')

from generator.read_roadmap import read_roadmap
from generator.write_json import write_json_flow_file

from collections import Counter

In [2]:
def main(subdirect, random_routes, random_vehicle_parameters):
    
    Road_map_Json = f"../Simulation_Results/{subdirect}/roadnet.json"
    Dir_FlowFile_Json = f"../Simulation_Results/{subdirect}/"
    FlowFile_Json = "flow.json"
    
    # if we want inflow / outflow from specific lanes, we can just overwrite roads_IN and roads_OUT
    roads_IN, roads_OUT, road_adj_to_IN = read_roadmap(Road_map_Json)
    
    # "Fixed" or random vehicle routes (e.g. main and side roads)
    # ratio: what percentage [0,1] of all possible inflow-outflow combinations will have vehicles
    # effective_interval: what is the effective interval between 2 vehicles on a road (even with multiple routes starting from the same road)
    if random_routes:
        # what percentage [0,1] of all possible inflow and outflow combinations will be considered
        parameters = {
            "traffic_ratio": 0.6,
            "interval": 50.0
        }
    else:
        main_roads = {"road_0_1_0", "road_1_1_2", "road_4_1_0", "road_5_1_2"}
        parameters = {
            "main_roads": main_roads,
            # the remaining roads are side roads
            "side_roads": roads_IN.union(roads_OUT).difference(main_roads),
            "mtm": {"ratio": 0.8, "effective_interval": 3},
            "mts": {"ratio": 0, "effective_interval": 3},
            "sts": {"ratio": 0.7, "effective_interval": 15},
            "stm": {"ratio": 0, "effective_interval": 15}
        }
        
            

    
    # Fixed or random vehicle parameters
    if random_vehicle_parameters:
        write_json_flow_file(
            vehLen =  5.0,
            vehWidth =  2.0,
            vehMaxPosAcc = 3.0,
            vehMaxNegAcc = 4.5,
            vehUsualPosAcc = 2.0,
            vehUsualNegAcc = 3.5,
            vehMinGap = 2.5, # 2.5
            vehMaxSpeed =  16.67,
            vehHeadwayTime = 10,
            parameters = parameters,
            roads_IN = roads_IN,
            roads_OUT = roads_OUT,
            road_adj_to_IN = road_adj_to_IN,
            directory = Dir_FlowFile_Json,
            FlowFile = FlowFile_Json,
            random_routes = random_routes,
            random_vehicle_parameters = random_vehicle_parameters
        )
        
    else:
        write_json_flow_file(
            vehLen =  5,
            vehWidth =  2,
            roads_IN = roads_IN,
            roads_OUT = roads_OUT,
            road_adj_to_IN = road_adj_to_IN,
            parameters = parameters,
            directory = Dir_FlowFile_Json,
            FlowFile = FlowFile_Json,
            random_routes = random_routes,
            random_vehicle_parameters = random_vehicle_parameters
        )
            

In [3]:
if __name__ == "__main__":
    
    # random vehicle routes
    random_routes = False
    
    # random vehicle parameters
    random_vehicle_parameters = False
    
    # sub directory of the roadnet file
    subdirect = "Manhattan"
    
    main(subdirect, random_routes, random_vehicle_parameters)

KeyError: 8